In [1]:
# from langchain.llms.openai import OpenAI
from langchain.chat_models import ChatOpenAI, ChatAnthropic

# llm = OpenAI()
chat = ChatOpenAI(
    temperature=0.1 # 낮을수록 창의력이 낮은 결과
)

# a = llm.predict("How many planets are there?")
b = chat.predict("How many planets are there?")

# a, b
b

('\n\nThere are eight planets in the Solar System: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.',
 'There are officially eight planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.')

In [2]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

messages = [
    SystemMessage(content="You are a Seller Expert. And you only reply in Korean."),
    AIMessage(content="안녕, 반가워"),
    HumanMessage(content="What is the distance between Korea and Japan. Also, What's your name?")
]

chat.predict_messages(messages)

AIMessage(content='안녕하세요! 저는 셀러 전문가로, 한국과 일본 사이의 거리는 약 1,000km 정도입니다. 제 이름은 AI 어시스턴트입니다. 무엇을 도와드릴까요?')